In [1]:
import pandas as pd
import re
from nltk.cluster import KMeansClusterer
import nltk
import numpy as np
import spacy
from sklearn.cluster import KMeans
from tqdm import tqdm
import requests

In [2]:
# load file

locations = pd.read_csv('./data/librettos_theaters.csv', delimiter='\t')
locations['id'] = [int(re.findall(r'\d+', s)[0]) for s in locations.file_name]
locations = locations.drop(columns=['Unnamed: 0'])
locations.sample(5)

,file_name,title,date,coperta,pot_city_name,coperta_appended,city_name,latitude,longitude,pot_city_name_fuzzy,...,predicted_cluster_method_1,inferred_latitude,inferred_longitude,title_opera_method_1,title_opera_method_2,title_vec_method_1,predicted_title_cluster_method_1,composer,inferred_title,title_mediawiki_pageid
916,data_984.json,Pietro d'Abano. melodramma serio in 3 atti da ...,1856,"['PIETRO', 'DABANO', '‘Lone', 'SEO', '3', '.',...","['reggio', 'bologna', 'padova', 'reggio', 'bol...",1,reggio,44.69825,10.63125,"['reggio', 'bologna', 'padova', 'reggio', 'bol...",...,3,45.4337096,12.3338572,Pietro d'Abano,Pietro d'Abano,[ 3.9548185 1.8186283 3.3242702 -2.811337...,689,Not found,Pietro d'Abano,384874
83,data_240.json,"Fernando Cortez, melodramma serio in tre atti....",1850,"['ROL', '.', '0432', '.', '0', 'Personngai', '...",['reggio'],1,reggio,44.69825,10.63125,['reggio'],...,3,45.4337096,12.3338572,Fernando Cortez,Fernando Cortez,[ 3.48391 -0.17130496 3.8326225 -3.411186...,160,Not found,Fernando Cortez,1219096
959,data_72.json,Antioco. Dramma per musica nel Teatro San Cass...,1658,"['a', 'eS', 'ANTIOCO', '.', 'PANY', '.', '297L...","['venetia', 'venezia', 'venetia']",0,venetia,45.43713,12.33265,"['venetia', 'venezia', 'venetia']",...,62,39.2252916,9.12122,Antioco,Antioco,[ 3.1054902 4.644814 4.211316 -2.08...,268,Not found,Antioco,224836
717,data_132.json,Caio Marzio Coriolano dramma per musica da rap...,1747,"['CAIO', 'MARZIO', 'CORIOLANO', 'DRAMMA', 'PER...","['roma', 'roma', 'anzio', 'venezia', 'venetia'...",1,roma,41.89193,12.51133,"['roma', 'roma', 'anzio', 'venezia', 'venetia'...",...,16,45.776982149999995,8.882520992545153,Caio Marzio Coriolano,Caio Marzio Coriolano,[ 6.8437595 1.682654 2.3513553 -3.427437...,265,Not found,Caio Marzio Coriolano,1242924
782,data_528.json,"I virtuosi di teatro, dramma giocoso di un sol...",1818,"['1', 'VIRTUOSI', 'DI', 'TEATRO', 'DRAMA', 'Gl...",[],1,0,0.00000,0.00000,[],...,1,Not found,Not found,I virtuosi di teatro,Not found,[-2.4034619e+00 3.3036885e+00 2.0615077e+00 ...,383,Not found,I virtuosi di teatro,1157558


In [10]:
# method 1
nlp = spacy.load("it_core_news_sm")

locations['occasion_method_1'] = [[ent.text
                                  for ent in nlp(title).ents
                                  if (ent.label_ == 'MISC' or ent.label_ == 'ORG')]
                                  for title in locations.title 
                                  ]
for loc in locations['occasion_method_1']:
    print(loc, len(loc) )

['Cesare in Egitto'] 1
[] 0
[] 0
['SS'] 1
[] 0
['S.M.I.R.A. &'] 1
[] 0
['Olimpiade'] 1
['The music by Martini'] 1
[] 0
['Montecchi'] 1
['nobiliss.me dame', "nobiliss.mi cavalieri di detta citta'"] 2
['LL', 'AA', 'Serenissimo'] 3
[] 0
[] 0
[] 0
[] 0
['maestà del Re nostro invitto'] 1
['Musica'] 1
[] 0
[] 0
[] 0
[] 0
[] 0
["Carnevale dell'Anno 1724", 'Maestà di Giacomo 3', 'Inghilterra &c.'] 3
[] 0
[] 0
['Amleto'] 1
['Morlacchi'] 1
[] 0
[] 0
[] 0
[] 0
['Siroe re di Persia'] 1
[] 0
[] 0
[] 0
["La caduta dell'ultimo Gran Visire", 'Drama'] 2
[] 0
["L'Aluilda Regina de Goti"] 1
["Basilio ré d'Oriente"] 1
[] 0
['Medea'] 1
[] 0
['Carnevale 1751'] 1
[] 0
['Ciro'] 1
['Imperial Regio Teatro di Vienna'] 1
[] 0
["L'Atanagilda Regina di Gottia"] 1
[] 0
['Augustissimi Sovrani', 'Imperator Carlo 6'] 2
[] 0
['Russie', 'Imperial Teatro di St.'] 2
[] 0
['La vergine del sole'] 1
[] 0
['Tron'] 1
[] 0
["Carnovale dell'Anno 1722"] 1
['Cherusci'] 1
[] 0
[] 0
['Faramondo', "Carnovale dell'Anno 1720", 'Maestà d

['LL', 'AA', 'Serenissimo'] 3
[] 0
[] 0
['Imperial Regia Corte', 'Sacra Reale Maestà'] 2
[] 0
['Longobardi', 'Drama'] 2
[] 0
[] 0
[] 0
['Jefte'] 1
[] 0
[] 0
[] 0
[] 0
['Arianna del sig'] 1
[] 0
[] 0
[] 0
['La maga Circe'] 1
[] 0
[] 0
[] 0
['Temistocle'] 1
['Sepolcro di Nostro Signore', 'Domenica delle Palme'] 2
[] 0
[] 0
[] 0
[] 0
[] 0
[] 0
[] 0
[] 0
['Erranti di Brescia'] 1
[] 0
["Chiaui d'oro"] 1
[] 0
[] 0
['A.R.'] 1
[] 0
[] 0
[] 0
['Nuovo Teatro di Padova'] 1
['Tauride'] 1
[] 0
["Colleggiata de' Ss"] 1
[] 0
['Asilo', 'Scherzo'] 2
[] 0
['Calvario componimento sacro'] 1
['Cerveteri &c. Posto'] 1
['Eccellenza del Sig'] 1
[] 0
['S.R.'] 1
['Musica', 'Primavera 1837'] 2
[] 0
[] 0
['Re musicali præclaro'] 1
["All'Illustrissimo"] 1
[] 0
['N.N.'] 1
['Eraclea tragicomedia'] 1
[] 0
['Armida'] 1
[] 0
['Dartula'] 1
['Drama'] 1
[] 0
[] 0
[] 0
[] 0
[] 0
[] 0
[] 0
['Piis virginibus', 'Nosocomio Nuncupato Pauperum Derelictorum', 'Modos'] 3
['Drama', 'Eccellenza', 'Signor Gio:'] 3
['Il convitato di p

In [ ]:
time = r"in |anno |l'anno|il |l'autunno|la primavera|l'estate|l'inverno|dell'anno|il novembre|l'auttuno|ne' mesi di|nel mese di|per la prima opera del|per la fiera|la fiera|nel prossimo|nel |nella |nell'occasione|con l'occasione|nella stagione di|la stagione|nell'està|in occasione|per la |per il |nel presente|del \d+ |carnovale|\d+ |\, "
'carnovale', 'olimpiade', 'fiera'